# Triplet Neural Network

## Transfer Learning

In [8]:
from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras import backend as K

In [2]:
input1 = Input(shape=(4096,), name='anchor')
input2 = Input(shape=(4096,), name='positive')
input3 = Input(shape=(4096,), name='negative')

W0718 12:27:17.306177 140060682143552 deprecation_wrapper.py:119] From /home/vahe/anaconda3/envs/similarity/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [3]:
shared_layer = Dense(100, name='shared')
output1 = shared_layer(input1)
output2 = shared_layer(input2)
output3 = shared_layer(input3)

W0718 12:27:19.379912 140060682143552 deprecation_wrapper.py:119] From /home/vahe/anaconda3/envs/similarity/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [4]:
merged_vector = concatenate([output1, output2, output3], axis=-1, name='merged_layer')

In [5]:
model = Model(inputs=[input1,input2,input3], outputs=merged_vector)

W0718 12:27:25.853374 140060682143552 deprecation_wrapper.py:119] From /home/vahe/anaconda3/envs/similarity/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [6]:
def triplet_loss(y_dummy, y_concatenated, margin=0.5):
    total_length = y_concatenated.shape.as_list()[-1]
    y_anchor = y_concatenated[:,:int(total_length*1/3)]
    y_positive = y_concatenated[:,int(total_length*1/3):int(total_length*2/3)]
    y_negative = y_concatenated[:,int(total_length*2/3):]
    
    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(y_anchor-y_positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(y_anchor-y_negative),axis=1)

    # compute loss
    basic_loss = pos_dist-neg_dist+margin
    loss = K.maximum(basic_loss,0.0)
 
    return loss

In [9]:
model.compile(optimizer='SGD', loss=triplet_loss)

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor (InputLayer)             (None, 4096)         0                                            
__________________________________________________________________________________________________
positive (InputLayer)           (None, 4096)         0                                            
__________________________________________________________________________________________________
negative (InputLayer)           (None, 4096)         0                                            
__________________________________________________________________________________________________
shared (Dense)                  (None, 100)          409700      anchor[0][0]                     
                                                                 positive[0][0]                   
          

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
import copy
import numpy as np

from keras.engine.network import Network
from keras.engine.base_layer import Layer
from keras.engine.training_utils import collect_metrics
from keras.engine.training_utils import check_array_length_consistency
from keras.engine.training_utils import check_loss_and_target_compatibility
from keras.engine.training_utils import standardize_class_weights
from keras.engine.training_utils import standardize_input_data
from keras.engine.training_utils import standardize_sample_weights
from keras.engine.training_utils import standardize_weights
from keras.engine.training_utils import weighted_masked_objective
from keras.engine import training_arrays
from keras.engine import training_generator
from keras import backend as K
from keras import optimizers
from keras import losses
from keras import metrics as metrics_module
from keras.utils.generic_utils import slice_arrays
from keras.utils.generic_utils import to_list
from keras.utils.generic_utils import unpack_singleton
from keras.legacy import interfaces

In [ ]:
from keras.models import Model

class MyModel(Model):
    def compile1(self, optimizer,
                loss=None,
                metrics=None,
                loss_weights=None,
                sample_weight_mode=None,
                weighted_metrics=None,
                target_tensors=None,
                **kwargs):
        """Configures the model for training.
        # Arguments
            optimizer: String (name of optimizer) or optimizer instance.
                See [optimizers](/optimizers).
            loss: String (name of objective function) or objective function.
                See [losses](/losses).
                If the model has multiple outputs, you can use a different loss
                on each output by passing a dictionary or a list of losses.
                The loss value that will be minimized by the model
                will then be the sum of all individual losses.
            metrics: List of metrics to be evaluated by the model
                during training and testing.
                Typically you will use `metrics=['accuracy']`.
                To specify different metrics for different outputs of a
                multi-output model, you could also pass a dictionary,
                such as `metrics={'output_a': 'accuracy'}`.
            loss_weights: Optional list or dictionary specifying scalar
                coefficients (Python floats) to weight the loss contributions
                of different model outputs.
                The loss value that will be minimized by the model
                will then be the *weighted sum* of all individual losses,
                weighted by the `loss_weights` coefficients.
                If a list, it is expected to have a 1:1 mapping
                to the model's outputs. If a dict, it is expected to map
                output names (strings) to scalar coefficients.
            sample_weight_mode: If you need to do timestep-wise
                sample weighting (2D weights), set this to `"temporal"`.
                `None` defaults to sample-wise weights (1D).
                If the model has multiple outputs, you can use a different
                `sample_weight_mode` on each output by passing a
                dictionary or a list of modes.
            weighted_metrics: List of metrics to be evaluated and weighted
                by sample_weight or class_weight during training and testing.
            target_tensors: By default, Keras will create placeholders for the
                model's target, which will be fed with the target data during
                training. If instead you would like to use your own
                target tensors (in turn, Keras will not expect external
                Numpy data for these targets at training time), you
                can specify them via the `target_tensors` argument. It can be
                a single tensor (for a single-output model), a list of tensors,
                or a dict mapping output names to target tensors.
            **kwargs: When using the Theano/CNTK backends, these arguments
                are passed into `K.function`.
                When using the TensorFlow backend,
                these arguments are passed into `tf.Session.run`.
        # Raises
            ValueError: In case of invalid arguments for
                `optimizer`, `loss`, `metrics` or `sample_weight_mode`.
        """
        self.optimizer = optimizers.get(optimizer)
        self.loss = loss or []
        self.metrics = metrics or []
        self.loss_weights = loss_weights
        self.sample_weight_mode = sample_weight_mode
        self.weighted_metrics = weighted_metrics

        if not self.built:
            # Model is not compilable because
            # it does not know its number of inputs
            # and outputs, nor their shapes and names.
            # We will compile after the first
            # time the model gets called on training data.
            return
        self._is_compiled = True

        
        loss_function = losses.get(loss)
        loss_functions = [loss_function for _ in range(len(self.outputs))]
        self.loss_functions = loss_functions
        weighted_losses = [
            weighted_masked_objective(fn) for fn in loss_functions]
        skip_target_indices = []
        skip_target_weighing_indices = []
        self._feed_outputs = []
        self._feed_output_names = []
        self._feed_output_shapes = []
        self._feed_loss_fns = []
        for i in range(len(weighted_losses)):
            if weighted_losses[i] is None:
                skip_target_indices.append(i)
                skip_target_weighing_indices.append(i)
        # Prepare output masks.
        masks = self.compute_mask(self.inputs, mask=None)
        if masks is None:
            masks = [None for _ in self.outputs]
        masks = to_list(masks)
        
         # Prepare loss weights.
        if loss_weights is None:
            loss_weights_list = [1. for _ in range(len(self.outputs))]
        elif isinstance(loss_weights, dict):
            for name in loss_weights:
                if name not in self.output_names:
                    raise ValueError('Unknown entry in loss_weights '
                                     'dictionary: "' + name + '". '
                                     'Only expected the following keys: ' +
                                     str(self.output_names))
            loss_weights_list = []
            for name in self.output_names:
                loss_weights_list.append(loss_weights.get(name, 1.))
        elif isinstance(loss_weights, list):
            if len(loss_weights) != len(self.outputs):
                raise ValueError('When passing a list as loss_weights, '
                                 'it should have one entry per model output. '
                                 'The model has ' + str(len(self.outputs)) +
                                 ' outputs, but you passed loss_weights=' +
                                 str(loss_weights))
            loss_weights_list = loss_weights
        else:
            raise TypeError('Could not interpret loss_weights argument: ' +
                            str(loss_weights) +
                            ' - expected a list of dicts.')
            
        # Prepare targets of model.
        self.targets = []
        self._feed_targets = []
#         if target_tensors is not None:
#             if isinstance(target_tensors, list):
#                 if len(target_tensors) != len(self.outputs):
#                     raise ValueError(
#                         'When passing a list as `target_tensors`, '
#                         'it should have one entry per model output. '
#                         'The model has ' + str(len(self.outputs)) +
#                         ' outputs, but you passed target_tensors=' +
#                         str(target_tensors))
#             elif isinstance(target_tensors, dict):
#                 for name in target_tensors:
#                     if name not in self.output_names:
#                         raise ValueError('Unknown entry in `target_tensors` '
#                                          'dictionary: "' + name + '". '
#                                          'Only expected the following keys: ' +
#                                          str(self.output_names))
#                 tmp_target_tensors = []
#                 for name in self.output_names:
#                     tmp_target_tensors.append(target_tensors.get(name, None))
#                 target_tensors = tmp_target_tensors
#             elif K.is_tensor(target_tensors):
#                 if len(self.outputs) != 1:
#                     raise ValueError('The model has ' + str(len(self.outputs)) +
#                                      ' outputs, but you passed a single tensor as '
#                                      '`target_tensors`. Expected a list or a dict '
#                                      'of tensors.')
#                 target_tensors = [target_tensors]
#             else:
#                 raise TypeError('Expected `target_tensors` to be a tensor, '
#                                 'a list of tensors, or dict of tensors, but got:',
#                                 target_tensors)
        
        for i in range(len(self.outputs)):
            if i in skip_target_indices:
                self.targets.append(None)
            else:
                shape = K.int_shape(self.outputs[i])
                name = self.output_names[i]
                if target_tensors is not None:
                    target = target_tensors[i]
                else:
                    target = None
                if target is None or K.is_placeholder(target):
                    if target is None:
                        target = K.placeholder(
                            ndim=len(shape),
                            name=name + '_target',
                            sparse=K.is_sparse(self.outputs[i]),
                            dtype=K.dtype(self.outputs[i]))
                    self._feed_targets.append(target)
                    self._feed_outputs.append(self.outputs[i])
                    self._feed_output_names.append(name)
                    self._feed_output_shapes.append(shape)
                    self._feed_loss_fns.append(self.loss_functions[i])
                else:
                    skip_target_weighing_indices.append(i)
                self.targets.append(target)

        # Prepare sample weights.
        sample_weights = []
        sample_weight_modes = []
        if isinstance(sample_weight_mode, dict):
            for name in sample_weight_mode:
                if name not in self.output_names:
                    raise ValueError('Unknown entry in '
                                     'sample_weight_mode dictionary: "' +
                                     name + '". '
                                     'Only expected the following keys: ' +
                                     str(self.output_names))
            for i, name in enumerate(self.output_names):
                if i in skip_target_weighing_indices:
                    weight = None
                    sample_weight_modes.append(None)
                else:
                    if name not in sample_weight_mode:
                        raise ValueError('Output "' + name +
                                         '" missing from sample_weight_modes '
                                         'dictionary')
                    if sample_weight_mode.get(name) == 'temporal':
                        weight = K.placeholder(ndim=2,
                                               name=name + '_sample_weights')
                        sample_weight_modes.append('temporal')
                    else:
                        weight = K.placeholder(ndim=1,
                                               name=name + '_sample_weights')
                        sample_weight_modes.append(None)
                sample_weights.append(weight)
        elif isinstance(sample_weight_mode, list):
            if len(sample_weight_mode) != len(self.outputs):
                raise ValueError('When passing a list as sample_weight_mode, '
                                 'it should have one entry per model output. '
                                 'The model has ' + str(len(self.outputs)) +
                                 ' outputs, but you passed '
                                 'sample_weight_mode=' +
                                 str(sample_weight_mode))
            for i in range(len(self.output_names)):
                if i in skip_target_weighing_indices:
                    weight = None
                    sample_weight_modes.append(None)
                else:
                    mode = sample_weight_mode[i]
                    name = self.output_names[i]
                    if mode == 'temporal':
                        weight = K.placeholder(ndim=2,
                                               name=name + '_sample_weights')
                        sample_weight_modes.append('temporal')
                    else:
                        weight = K.placeholder(ndim=1,
                                               name=name + '_sample_weights')
                        sample_weight_modes.append(None)
                sample_weights.append(weight)
        else:
            for i, name in enumerate(self.output_names):
                if i in skip_target_weighing_indices:
                    sample_weight_modes.append(None)
                    sample_weights.append(None)
                else:
                    if sample_weight_mode == 'temporal':
                        sample_weights.append(
                            K.placeholder(ndim=2,
                                          name=name + '_sample_weights'))
                        sample_weight_modes.append('temporal')
                    else:
                        sample_weights.append(
                            K.placeholder(ndim=1,
                                          name=name + '_sample_weights'))
                        sample_weight_modes.append(None)
        self.sample_weight_modes = sample_weight_modes
        self._feed_sample_weight_modes = []
        for i in range(len(self.outputs)):
            if i not in skip_target_weighing_indices:
                self._feed_sample_weight_modes.append(
                    self.sample_weight_modes[i])
        
        # Prepare metrics.
        self.metrics_names = ['loss']
        self.metrics_tensors = []

        # Compute total loss.
        total_loss = None
        with K.name_scope('loss'):
            for i in range(len(self.outputs)):
                if i in skip_target_indices:
                    continue
                y_true = self.targets[i]
                y_pred = self.outputs[i]
                
                weighted_loss = weighted_losses[i]
                sample_weight = sample_weights[i]
                mask = masks[i]
                loss_weight = loss_weights_list[i]
                
                with K.name_scope(self.output_names[i] + '_loss'):
                    output_loss = weighted_loss(y_true, y_pred,
                                                sample_weight, mask)
                   
                
                if len(self.outputs) > 1:
                    self.metrics_tensors.append(output_loss)
                    self.metrics_names.append(self.output_names[i] + '_loss')
                if total_loss is None:
                    total_loss = loss_weight * output_loss
                else:
                    total_loss += loss_weight * output_loss
            if total_loss is None:
                if not self.losses:
                    raise ValueError('The model cannot be compiled '
                                     'because it has no loss to optimize.')
                else:
                    total_loss = 0.

            # Add regularization penalties
            # and other layer-specific losses.
            for loss_tensor in self.losses:
                total_loss += loss_tensor
        
        
        # List of same size as output_names.
        # contains tuples (metrics for output, names of metrics).
        nested_metrics = collect_metrics(metrics, self.output_names)
        nested_weighted_metrics = collect_metrics(weighted_metrics,
                                                  self.output_names)
        self.metrics_updates = []
        self.stateful_metric_names = []
        self.stateful_metric_functions = []

In [ ]:
model1 = MyModel(inputs=[input1,input2,input3], outputs=merged_vector)
model1.compile1(optimizer='SGD', loss=triplet_loss)